In [159]:
from keras.models import load_model

saved_model_file = 'ptc_model.h5'
model = load_model(saved_model_file)

In [160]:
num_of_classes = 2
path_root = "/home/muskaanjain/ntu/Muskaan/graph_2D_CNN/datasets/"
dataset = "test"
definition = 14
n_channels = 5
dim_ordering = 'th' # channels first

In [161]:
from os.path import dirname, abspath, join
import sys
import os
import re
import numpy as np
from keras.utils import np_utils

sys.path.append('/home/muskaanjain/ntu')

print('========== loading labels ==========')

with open(path_root + 'classes/' + dataset + '/' + dataset + '_classes.txt', 'r') as f:
    ys = f.read().splitlines()
    ys = [int(elt) for elt in ys]

num_classes = len(list(set(ys)))
print('classes:', list(set(ys)))

print('converting to 0-based index')

if 0 not in list(set(ys)):
    if -1 not in list(set(ys)):
        ys = [y-1 for y in ys]
    else:
        ys = [1 if y==1 else 0 for y in ys]

print('classes:', list(set(ys)) )

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

print('========== loading tensors ==========')

train_path_read = path_root + 'hist/' + 'train' + '/embed_hist/'
test_path_read = path_root + 'hist/' + 'test' + '/embed_hist/'

train_file_names = [elt for elt in os.listdir(train_path_read)] # make sure the right files are selected
test_file_names = [elt for elt in os.listdir(test_path_read)] # make sure the right files are selected
train_file_names.sort(key=natural_keys)
test_file_names.sort(key=natural_keys)

print(len(train_file_names), len(test_file_names))
print(train_file_names[:5], test_file_names[:5])
print(train_file_names[-5:], test_file_names[-5:])

ys = [y for idx,y in enumerate(ys)]
print(len(train_file_names) + len(test_file_names) == len(ys))

print('converting labels to array')
ys = np.array(ys)

print('transforming integer labels into one-hot vectors')
ys = np_utils.to_categorical(ys, num_classes)

========== loading labels ==========
classes: [1, -1]
converting to 0-based index
classes: [0, 1]
========== loading tensors ==========
210 90
['0.npy', '1.npy', '2.npy', '3.npy', '4.npy'] ['0.npy', '1.npy', '2.npy', '3.npy', '4.npy']
['205.npy', '206.npy', '207.npy', '208.npy', '209.npy'] ['85.npy', '86.npy', '87.npy', '88.npy', '89.npy']
True
converting labels to array
transforming integer labels into one-hot vectors


In [162]:
train_tensors = []
test_tensors = []

for name in train_file_names:
    tensor = np.load(train_path_read + name)
    train_tensors.append(tensor[:n_channels,:,:])
    
for name in test_file_names:
    tensor = np.load(test_path_read + name)
    test_tensors.append(tensor[:n_channels,:,:])
    
train_tensors = np.array(train_tensors)
train_tensors = train_tensors.astype('float32')
train_tensors = np.rollaxis(train_tensors, 3, 1)
train_tensors = np.rollaxis(train_tensors, 3, 1)

test_tensors = np.array(test_tensors)
test_tensors = test_tensors.astype('float32')
test_tensors = np.rollaxis(test_tensors, 3, 1)
test_tensors = np.rollaxis(test_tensors, 3, 1)

print('train tensors shape:', train_tensors.shape)
print('test tensors shape:', test_tensors.shape)


print('========== getting image dimensions ==========')

img_rows, img_cols = int(train_tensors.shape[2]), int(train_tensors.shape[3])
input_shape = (int(train_tensors.shape[1]), img_rows, img_cols)   

print('input shape:', input_shape )

print('========== shuffling data ==========')

saved_dataset_idxs = np.load('/home/muskaanjain/ntu/Muskaan/gspannew/shuffled_graph_idxs.npy')
ys = ys[saved_dataset_idxs.tolist()]

#shuffled_idxs = random.sample(range(tensors.shape[0]), int(tensors.shape[0])) # sample w/o replct
#tensors = tensors[shuffled_idxs]
#ys = ys[shuffled_idxs]

train tensors shape: (210, 131, 131, 5)
test tensors shape: (90, 131, 131, 5)
========== getting image dimensions ==========
input shape: (131, 131, 5)
========== shuffling data ==========


In [163]:
train_test_split = 0.7
split_point = (int)(len(train_tensors))
x_train = np.array(train_tensors)
x_test = np.array(test_tensors)
        
y_train = np.array(ys[:split_point])
y_test = np.array(ys[split_point:])

print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))

True
True


In [164]:
model.evaluate(x_test, y_test)

90/90 [==============================] - 1s 16ms/step


[1.3733558575312297, 0.600000003973643]

In [165]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 129, 129, 64)      2944      
_________________________________________________________________
activation_17 (Activation)   (None, 129, 129, 64)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 62, 62, 96)        55392     
_________________________________________________________________
activation_18 (Activation)   (None, 62, 62, 96)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 31, 31, 96)        0         
__________

In [166]:
sys.path.append('/home/muskaanjain/git/deeplift')

import deeplift
from deeplift.layers import NonlinearMxtsMode
from deeplift.conversion import kerasapi_conversion as kc

revealcancel_model = kc.convert_model_from_saved_files(
                            h5_file=saved_model_file,
                            nonlinear_mxts_mode=NonlinearMxtsMode.RevealCancel)

nonlinear_mxts_mode is set to: RevealCancel
131
129
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
65
63
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
Heads-up: I assume softmax is the output layer, not an intermediate one; if it's an intermediate layer, please let me know and I will prioritise that use-case


In [167]:
from deeplift.util import compile_func
import numpy as np
from keras import backend as K

deeplift_model = revealcancel_model
deeplift_prediction_func = compile_func([deeplift_model.get_layers()[0].get_activation_vars()],
                                       deeplift_model.get_layers()[-1].get_activation_vars())
original_model_predictions = model.predict(x_test, batch_size=32)
converted_model_predictions = deeplift.util.run_function_in_batches(
                                input_data_list=[x_test],
                                func=deeplift_prediction_func,
                                batch_size=32,
                                progress_update=None)
print("difference in predictions:",np.max(np.array(converted_model_predictions)-np.array(original_model_predictions)))
assert np.max(np.array(converted_model_predictions)-np.array(original_model_predictions)) < 10**-5
predictions = converted_model_predictions

difference in predictions: 0.0


In [168]:
from keras import backend as K
import deeplift
from deeplift.util import get_integrated_gradients_function

revealcancel_func = revealcancel_model.get_target_contribs_func(find_scores_layer_idx=0, target_layer_idx=-2)

2
OneAndZeros


In [169]:
from collections import OrderedDict
method_to_task_to_scores = OrderedDict()

for method_name, score_func in [('revealcancel', revealcancel_func)]:
    print("Computing scores for:",method_name)
    method_to_task_to_scores[method_name] = {}
    for task_idx in range(num_of_classes):
        print("\tComputing scores for task: "+str(task_idx))
        scores = np.array(score_func(
                    task_idx=task_idx,
                    input_data_list=[x_test],
                    input_references_list=[np.zeros_like(x_test)],
                    batch_size=100,
                    progress_update=None))
        method_to_task_to_scores[method_name][task_idx] = scores

Computing scores for: revealcancel
	Computing scores for task: 0
	Computing scores for task: 1


In [170]:
scores.shape

(90, 131, 131, 5)

In [171]:
h_test = [] #corresponds to 5*131*131
graph_files  = os.listdir('/home/muskaanjain/ntu/version1/graphimg_with_ids/test/') 
for idx, name in enumerate(graph_files):
    graph = np.load('/home/muskaanjain/ntu/version1/graphimg_with_ids/test/' + name)
    h_test.append(graph)

address_test = np.empty([len(h_test), 5, 131, 131], dtype=int)
for graph in range(len(h_test)):
    for channel in range(n_channels):
        address_test[graph, channel, :, :] = (np.arange(1, (131*131)+1, 1)).reshape(131, 131)
    
address_test = np.array(address_test)
print(np.array(address_test).shape)
address_test = np.rollaxis(address_test, 3, 1)
address_test = np.rollaxis(address_test, 3, 1)
print(np.array(address_test).shape)


h_train = [] #corresponds to 5*131*131
graph_files  = os.listdir('/home/muskaanjain/ntu/version1/graphimg_with_ids/train/') 
for idx, name in enumerate(graph_files):
    graph = np.load('/home/muskaanjain/ntu/version1/graphimg_with_ids/train/' + name)
    h_train.append(graph)

address_train = np.empty([len(h_train), 5, 131, 131], dtype=int)
for graph in range(len(h_train)):
    for channel in range(n_channels):
        address_train[graph, channel, :, :] = (np.arange(1, (131*131)+1, 1)).reshape(131, 131)
    
address_train = np.array(address_train)
print(np.array(address_train).shape)
address_train = np.rollaxis(address_train, 3, 1)
address_train = np.rollaxis(address_train, 3, 1)
print(np.array(address_train).shape)

(90, 5, 131, 131)
(90, 131, 131, 5)
(210, 5, 131, 131)
(210, 131, 131, 5)


In [172]:
#Store the image indices which are predicted right or wrong separately for cats and dogs.
y_pred = (np.array(converted_model_predictions) >= 0.5)*1 + (np.array(converted_model_predictions) < 0.5)*0
predictions = np.argmax(y_pred, axis=1)
original = np.argmax(y_test, axis=1)

right_0_pred_idx = []
wrong_0_pred_idx = []
right_1_pred_idx = []
wrong_1_pred_idx = []

for i in range(len(y_test)):
    if(original[i] == 0):
        if(predictions[i] == 0):
            right_0_pred_idx.append(i)
        else:
            wrong_0_pred_idx.append(i)
    else:
        if(predictions[i] == 1):
            right_1_pred_idx.append(i)
        else:
            wrong_1_pred_idx.append(i)
            
print("Sanity check :")
print((len(right_0_pred_idx) + len(right_1_pred_idx) + len(wrong_0_pred_idx) + len(wrong_1_pred_idx)) == len(y_test))

print("0 predictions:: right : ", len(right_0_pred_idx), " wrong : ", len(wrong_0_pred_idx))
print("1 predictions:: right : ", len(right_1_pred_idx), " wrong : ", len(wrong_1_pred_idx))

Sanity check :
True
0 predictions:: right :  32  wrong :  17
1 predictions:: right :  22  wrong :  19


In [174]:
random_img_set1 = [right_0_pred_idx[4], right_0_pred_idx[10], right_0_pred_idx[2], right_0_pred_idx[0]]
random_img_set2 = [wrong_0_pred_idx[4], wrong_0_pred_idx[10], wrong_0_pred_idx[2], wrong_0_pred_idx[0]]
random_img_set3 = [right_1_pred_idx[7], right_1_pred_idx[17], right_1_pred_idx[2], right_1_pred_idx[3]]
random_img_set4 = [wrong_1_pred_idx[7], wrong_1_pred_idx[17], wrong_1_pred_idx[2], wrong_1_pred_idx[3]]

## Test 1: Important subgraphs in an image: 

In [175]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

ix = 0
n_task1 = 15

for i in random_img_set1:
    s = method_to_task_to_scores['revealcancel'][0][i]
    arr = s.ravel()
    top_nth_threshold = max(sorted(arr, reverse=True)[n_task1],0.0) #can be done before smoothing the scores
    thresholded_points = 1.0*(arr >= top_nth_threshold)
    thresholded_points = thresholded_points.reshape(131, 131, 5) #why 5 channels?
    imp_inp = thresholded_points*x_test[i]
    imp_subgraphs = thresholded_points*address_test[i]
    #top_ones = imp_subgraphs[np.nonzero(imp_subgraphs)]
    top_ones = []
    for channel in range(n_channels):
        a = imp_subgraphs[:, :, channel].ravel()
        nonzeros = a[np.nonzero(a)]
        nonzeros = nonzeros.astype(int)
        for x in nonzeros:
                top_ones += h[i][channel][x-1]
    top_ones = set(top_ones)      
    extensions = np.load('/home/muskaanjain/ntu/version1/extensions.npy')
    extensions = np.array(extensions)
    print("for graph: ", i)
    for j in top_ones:
        print(j-1, ": ", extensions[int(j-1)])
    
    print("\n\n")
    ix += 1

for graph:  8
257 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0')]
365 :  [(0, 1, '18', '18', '0'), (1, 2, '18', '18', '0')]



for graph:  40
232 :  [(0, 1, '16', '18', '0')]
233 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '16', '0')]
234 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '16', '0'), (1, 3, '18', '16', '0')]
13 :  [(0, 1, '11', '18', '0')]



for graph:  4
129 :  [(0, 1, '11', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (1, 4, '18', '18', '0')]
137 :  [(0, 1, '11', '18', '0'), (0, 2, '11', '18', '0')]
171 :  [(0, 1, '14', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0')]
76 :  [(0, 1, '11', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (5, 6, '18', '18', '0'), (6, 1, '18', '18', '0'), (5, 7, '18', '18', '0')]
27 :  [(0, 1, '11', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '16

In [176]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

ix = 0
n_task1 = 15

for i in random_img_set3:
    s = method_to_task_to_scores['revealcancel'][1][i]
    arr = s.ravel()
    top_nth_threshold = max(sorted(arr, reverse=True)[n_task1],0.0) #can be done before smoothing the scores
    thresholded_points = 1.0*(arr >= top_nth_threshold)
    thresholded_points = thresholded_points.reshape(131, 131, 5) #why 5 channels?
    imp_inp = thresholded_points*x_test[i]
    imp_subgraphs = thresholded_points*address_test[i]
    #top_ones = imp_subgraphs[np.nonzero(imp_subgraphs)]
    top_ones = []
    for channel in range(n_channels):
        a = imp_subgraphs[:, :, channel].ravel()
        nonzeros = a[np.nonzero(a)]
        nonzeros = nonzeros.astype(int)
        for x in nonzeros:
                top_ones += h[i][channel][x-1]
    top_ones = set(top_ones)      
    
    extensions = np.load('/home/muskaanjain/ntu/version1/extensions.npy')
    extensions = np.array(extensions)
    print("for graph: ", i)
    for j in top_ones:
        print(j-1, ": ", extensions[int(j-1)])
    
    print("\n\n")
    
    ix += 1

for graph:  31
226 :  [(0, 1, '14', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (0, 4, '14', '18', '0')]
169 :  [(0, 1, '14', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '16', '0'), (0, 5, '14', '18', '0')]
138 :  [(0, 1, '14', '16', '0')]
171 :  [(0, 1, '14', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0')]
141 :  [(0, 1, '14', '16', '0'), (0, 2, '14', '18', '0')]
50 :  [(0, 1, '11', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '16', '0'), (4, 6, '18', '18', '0'), (3, 6, '18', '16', '0')]
181 :  [(0, 1, '14', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (5, 6, '18', '16', '0')]
150 :  [(0, 1, '14', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '16', '0'), (2, 4, '18', '18', '0')]



for graph:  81
34 :  [(0, 1, '11', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'

## Test 2: Subgraphs important for one task than the other

In [177]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

ix = 0
n_task1 = 15

for i in random_img_set2:
    s = method_to_task_to_scores['revealcancel'][1][i] - method_to_task_to_scores['revealcancel'][0][i]
    arr = s.ravel()
    top_nth_threshold = max(sorted(arr, reverse=True)[n_task1],0.0) #can be done before smoothing the scores
    thresholded_points = 1.0*(arr >= top_nth_threshold)
    thresholded_points = thresholded_points.reshape(131, 131, 5) #why 5 channels?
    imp_inp = thresholded_points*x_test[i]
    imp_subgraphs = thresholded_points*address_test[i]
    #top_ones = imp_subgraphs[np.nonzero(imp_subgraphs)]
    top_ones = []
    for channel in range(n_channels):
        a = imp_subgraphs[:, :, channel].ravel()
        nonzeros = a[np.nonzero(a)]
        nonzeros = nonzeros.astype(int)
        for x in nonzeros:
                top_ones += h[i][channel][x-1]
    top_ones = set(top_ones)      
    
    extensions = np.load('/home/muskaanjain/ntu/version1/extensions.npy')
    extensions = np.array(extensions)
    print("for graph: ", i)
    for j in top_ones:
        print(j-1, ": ", extensions[int(j-1)])
    
    print("\n\n")
    
    ix += 1

for graph:  29
383 :  [(0, 1, '18', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (5, 6, '18', '18', '0'), (6, 7, '18', '18', '0'), (7, 8, '18', '18', '0'), (5, 9, '18', '18', '0')]
286 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (5, 6, '18', '18', '0'), (6, 7, '18', '18', '0'), (7, 8, '18', '16', '0'), (5, 9, '18', '18', '0')]
5 :  [(0, 1, '3', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0')]
6 :  [(0, 1, '3', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0')]
241 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '16', '0'), (2, 4, '18', '18', '0')]
350 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (3, 5, '18', '18', '0')]



for graph:  43



for graph:  13
259 :  

In [178]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

ix = 0
n_task1 = 15

for i in random_img_set2:
    s = method_to_task_to_scores['revealcancel'][0][i] - method_to_task_to_scores['revealcancel'][1][i]
    arr = s.ravel()
    top_nth_threshold = max(sorted(arr, reverse=True)[n_task1],0.0) #can be done before smoothing the scores
    thresholded_points = 1.0*(arr >= top_nth_threshold)
    thresholded_points = thresholded_points.reshape(131, 131, 5) #why 5 channels?
    imp_inp = thresholded_points*x_test[i]
    imp_subgraphs = thresholded_points*address_test[i]
    #top_ones = imp_subgraphs[np.nonzero(imp_subgraphs)]
    top_ones = []
    for channel in range(n_channels):
        a = imp_subgraphs[:, :, channel].ravel()
        nonzeros = a[np.nonzero(a)]
        nonzeros = nonzeros.astype(int)
        for x in nonzeros:
                top_ones += h[i][channel][x-1]
    top_ones = set(top_ones)      
    
    extensions = np.load('/home/muskaanjain/ntu/version1/extensions.npy')
    extensions = np.array(extensions)
    print("for graph: ", i)
    for j in top_ones:
        print(j-1, ": ", extensions[int(j-1)])
    
    print("\n\n")
    
    ix += 1

for graph:  29
257 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0')]
292 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (5, 6, '18', '18', '0'), (6, 7, '18', '18', '0'), (7, 8, '18', '18', '0'), (8, 9, '18', '16', '0'), (8, 10, '18', '18', '0')]
327 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (5, 6, '18', '18', '0'), (6, 7, '18', '18', '0'), (3, 8, '18', '18', '0')]
328 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (5, 6, '18', '18', '0'), (6, 7, '18', '18', '0'), (2, 8, '18', '18', '0')]
332 :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (5, 6, '18', '18', '0'), (4, 7, '18', '18', 

## Important subgraphs for a class 

In [179]:
# For class 0
print("Total correctly predicted images for class 0: ", len(right_0_pred_idx))
print("\n\n")
subgraphs_score = np.zeros(len(extensions))
for img in right_0_pred_idx:
    s = method_to_task_to_scores['revealcancel'][0][img]
    arr = s.ravel()
    top_nth_threshold = max(sorted(arr, reverse=True)[n_task1],0.0) #can be done before smoothing the scores
    thresholded_points = 1.0*(arr >= top_nth_threshold)
    thresholded_points = thresholded_points.reshape(131, 131, 5) #why 5 channels?
    imp_inp = thresholded_points*x_test[img]
    imp_subgraphs = thresholded_points*address_test[img]
    #top_ones = imp_subgraphs[np.nonzero(imp_subgraphs)]
    top_ones = []
    for channel in range(n_channels):
        a = imp_subgraphs[:, :, channel].ravel()
        nonzeros = a[np.nonzero(a)]
        nonzeros = nonzeros.astype(int)
        for x in nonzeros:
                top_ones += h[img][channel][x-1]
    top_ones = set(top_ones)      
    for top in top_ones:
        subgraphs_score[top-1] += 1
    ix += 1
    
top_percent = 1
top_score_value = np.percentile(subgraphs_score, 100 - top_percent)
#print(top_filters_value)

top_subgraphs = []
for i in range(len(subgraphs_score)):
    if(subgraphs_score[i]>=top_score_value):
        top_subgraphs.append(i)

for i in top_subgraphs:
    print("subgraph id: ", i, " score: ", subgraphs_score[i])
    print("pattern : ", extensions[i])
    
#print(sorted(filter_score, reverse=True))

Total correctly predicted images for class 0:  32



subgraph id:  239  score:  13.0
pattern :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0')]
subgraph id:  252  score:  8.0
pattern :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '16', '0')]
subgraph id:  364  score:  12.0
pattern :  [(0, 1, '18', '18', '0')]
subgraph id:  365  score:  9.0
pattern :  [(0, 1, '18', '18', '0'), (1, 2, '18', '18', '0')]
subgraph id:  409  score:  7.0
pattern :  [(0, 1, '18', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (2, 4, '18', '18', '0')]


In [180]:
# For class 1
print("Total correctly predicted images for class 1: ", len(right_1_pred_idx))
print("\n\n")

subgraphs_score = np.zeros(len(extensions))
for img in right_1_pred_idx:
    s = method_to_task_to_scores['revealcancel'][1][img]
    arr = s.ravel()
    top_nth_threshold = max(sorted(arr, reverse=True)[n_task1],0.0) #can be done before smoothing the scores
    thresholded_points = 1.0*(arr >= top_nth_threshold)
    thresholded_points = thresholded_points.reshape(131, 131, 5) #why 5 channels?
    imp_inp = thresholded_points*x_test[img]
    imp_subgraphs = thresholded_points*address_test[img]
    #top_ones = imp_subgraphs[np.nonzero(imp_subgraphs)]
    top_ones = []
    for channel in range(n_channels):
        a = imp_subgraphs[:, :, channel].ravel()
        nonzeros = a[np.nonzero(a)]
        nonzeros = nonzeros.astype(int)
        for x in nonzeros:
                top_ones += h[img][channel][x-1]
    top_ones = set(top_ones)      
    for top in top_ones:
        subgraphs_score[top-1] += 1
    ix += 1
    
top_percent = 1
top_score_value = np.percentile(subgraphs_score, 100 - top_percent)
#print(top_filters_value)

top_subgraphs = []
for i in range(len(subgraphs_score)):
    if(subgraphs_score[i]>=top_score_value):
        top_subgraphs.append(i)

for i in top_subgraphs:
    print("subgraph id: ", i, " score: ", subgraphs_score[i])
    print("pattern : ", extensions[i])
    
#print(sorted(filter_score, reverse=True))

Total correctly predicted images for class 1:  22



subgraph id:  118  score:  6.0
pattern :  [(0, 1, '11', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '18', '0'), (4, 5, '18', '18', '0'), (1, 6, '18', '18', '0'), (2, 3, '18', '16', '0')]
subgraph id:  234  score:  7.0
pattern :  [(0, 1, '16', '18', '0'), (1, 2, '18', '16', '0'), (1, 3, '18', '16', '0')]
subgraph id:  240  score:  6.0
pattern :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '16', '0')]
subgraph id:  241  score:  13.0
pattern :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '16', '0'), (2, 4, '18', '18', '0')]
subgraph id:  245  score:  6.0
pattern :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (2, 3, '18', '18', '0'), (3, 4, '18', '16', '0'), (3, 5, '18', '18', '0')]
subgraph id:  363  score:  10.0
pattern :  [(0, 1, '16', '18', '0'), (1, 2, '18', '18', '0'), (1, 3, '18', '18', '0')]


In [181]:
print(classification_report(original, predictions))  #tp / (tp + fp) : precision
                                                     #tp / (tp + fn) : recall
                                                     #f1_score :  2 * (precision * recall) / (precision + recall)


              precision    recall  f1-score   support

           0       0.63      0.65      0.64        49
           1       0.56      0.54      0.55        41

    accuracy                           0.60        90
   macro avg       0.60      0.59      0.59        90
weighted avg       0.60      0.60      0.60        90



In [ ]:
#precision : out of predicted positives how many are true positives
#recall : out of true positives how many are predicted positives

#'macro':
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

#'weighted':
#Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.